In [2]:
!pip install -r requirements.txt

In [3]:
from datasets import load_dataset

In [23]:
dataset=load_dataset('JayalekshmiGopakumar/doclaynetTextClassDataset')

Found cached dataset parquet (/root/.cache/huggingface/datasets/JayalekshmiGopakumar___parquet/JayalekshmiGopakumar--doclaynetTextClassDataset-45d27921b1a0e2d8/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 691
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 64
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 49
    })
})

In [33]:
categories=dataset['train'].unique('label')

In [49]:
from transformers import AutoTokenizer

model_id = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [50]:
# Set the Falcon tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [57]:
from random import randint


# custom instruct prompt start
prompt_template = f"Classify the following text into one of the labels: {{categories}}\n text :{{text}}\n label : {{label}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["prompt"] = prompt_template.format(label=sample["label"], text=sample["text"][:2000],categories=categories,eos_token=tokenizer.eos_token)
    return sample

# apply prompt template per sample
train_dataset = dataset["train"].map(template_dataset, remove_columns=list(dataset["train"].features))
val_dataset = dataset["validation"].map(template_dataset, remove_columns=list(dataset["validation"].features))
test_dataset = dataset["test"].map(template_dataset, remove_columns=list(dataset["test"].features))


Map:   0%|          | 0/691 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [58]:
print(train_dataset[randint(0, len(dataset))]["prompt"])

Classify the following text into one of the labels: ['financial_reports', 'manuals', 'government_tenders', 'laws_and_regulations', 'scientific_articles', 'patents']
 text :Notes to the consolidated financial statements of Aegon N.V.  Note 50 270 50 Summary of total financial assets and financial liabilities at fair value through profit or loss Investments for general account Investments for account of policyholders Investment contracts for account of policyholders Derivatives Borrowings  The table that follows summarizes the carrying amounts of financial assets and financial liabilities that are classified as at fair value  through profit or loss  with appropriate distinction between those financial assets and financial liabilities held for trading and those  that  upon initial recognition  were designated as at fair value through profit or loss. The Group manages certain portfolios on a total return basis which have been designated at fair value through profit or loss.  This includes 

In [59]:
 # tokenize and chunk dataset
lm_train_dataset = train_dataset.map(
    lambda sample: tokenizer(sample["prompt"]), batched=True, batch_size=24, remove_columns=list(train_dataset.features)
)
lm_val_dataset = val_dataset.map(
    lambda sample: tokenizer(sample["prompt"]), batched=True, remove_columns=list(val_dataset.features)
)

lm_test_dataset = test_dataset.map(
    lambda sample: tokenizer(sample["prompt"]), batched=True, remove_columns=list(test_dataset.features)
)

# Print total number of samples
print(f"Total number of train samples: {len(lm_train_dataset)}")

Map:   0%|          | 0/691 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Total number of train samples: 691


In [60]:
# Save processed dataset to S3
bucket = "accelex-datasets"
directory = "doclaynet-processed"

lm_train_dataset.save_to_disk(f"s3://{bucket}/{directory}/train")
lm_val_dataset.save_to_disk(f"s3://{bucket}/{directory}/validation")
lm_test_dataset.save_to_disk(f"s3://{bucket}/{directory}/test")

Saving the dataset (0/1 shards):   0%|          | 0/691 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/64 [00:00<?, ? examples/s]